In [236]:
import math

## Graphes de calculs: les bases

Nous montrons ici comment on implémente un graphe de calcul et comment ce dernier peut être utilisé pour calculer des gradients de fonctions à plusieurs variables (variables réelles simples pour l'instant)

In [237]:
class Noeud:
    """
    Cette classe permet de réaliser le forward pass et le backward pass

    La structure de cette classe est simple: on définiti des méthodes réalisant l'exponentiation, l'addition la multiplication
    et qui instancie à chaque fois un noeud résultant avec pour attributs:
    - Une valeur
    - Des parents
    - Un gradient
    - Une fonction privée _backward attribut des noeuds créés par des opérations élémentaires

    Note: 
    Les fonctions _backward ainsi définies seront appelées par la méthode backward 
    """
    def __init__(self, valeur, parents=[]):
        self.valeur = valeur
        self.parents = parents

        self.grad = 0
        self._backward = lambda : None 

    def __add__(self, b):
        c = Noeud(self.valeur + b.valeur, parents=[self, b])

        def _backward():
            # On identifie self à a, on calcule donc le gradient pour a et b 
            self.grad += 1 * c.grad # += au cas ou les parents auraient plusieurs enfants
            b.grad += 1 * c.grad
        
        c._backward = _backward

        return c

    def __mul__(self, b):
        c = Noeud(self.valeur * b.valeur, parents=[self, b])

        def _backward():
            self.grad += b.valeur * c.grad
            b.grad += self.valeur * c.grad
        
        c._backward = _backward

        return c

    def exp(self):
        c = Noeud(math.exp(self.valeur), parents=[self])

        def _backward():
            self.grad += c.valeur * c.grad
        
        c._backward = _backward

        return c
    
    def clear_grads(self):
        a_visiter = self.parents.copy()
        
        while not len(a_visiter) == 0:
            s = a_visiter.pop()
            s.grad = 0
        
        for parent in s.parents:
            if parent not in a_visiter:
                a_visiter.append(parent)

    def backward(self):
        # remettre à 0 les gradients
        self.clear_grads()

        dejaVu = set()
        L = []

        def parcours(s):
            dejaVu.add(s)
            
            for parent in s.parents:
                if parent not in dejaVu:
                    parcours(parent)

                # fin traitement de s
            L.append(s)
                
        parcours(self)

        self.grad = 1 # Il s'agit du gradient de la sortie par rapport à elle même 
        for s in reversed(L):
            s._backward()

# Notes importantes:
---

**Utilisation de += dans _backward**
- L'opérateur += est utilisé pour mettre à jour le gradient de chaque parent (self et b ici) pendant la rétropropagation. La fonction _backward de c est définie pour augmenter le gradient de chaque parent par c.grad (le gradient du nœud actuel par rapport à la sortie du graphe) multiplié par la dérivée de la valeur de c par rapport à la valeur du parent, qui est 1 dans ce cas d'addition.

- Pour self et b, le gradient est augmenté (+=) parce que chaque nœud peut avoir plusieurs enfants contribuant à son gradient total. L'opération += assure que chaque contribution (de chaque enfant dans le graphe) est cumulativement ajoutée au gradient du nœud parent.

- Cette mise à jour est essentielle pour la rétropropagation, permettant à l'information du gradient de se propager à travers le graphe depuis la sortie vers l'entrée, pour que chaque nœud puisse ajuster sa valeur pour minimiser la fonction de coût lors de l'entraînement.
En résumé, le += permet d'accumuler les gradients venant de différents chemins dans le graphe de calcul, ce qui est crucial pour la mise à jour correcte des poids dans les réseaux de neurones lors de la rétropropagation.

**Fonctionnement de _backward()**
- Lorsqu'on définit une fonction `_backward` à l'intérieur d'une méthode comme `__add__` ou `__mul__`, cette fonction est une fermeture (closure en anglais) qui capture et retient les variables locales de la méthode dans laquelle elle a été définie. En Python, une fermeture permet à une fonction interne de se souvenir de l'état de son environnement lorsque la fonction a été créée, même après que le bloc de code extérieur a été exécuté.

- Dans notre cas, lorsque on définit `_backward` à l'intérieur de `__add__` ou `__mul__`, les variables `self`, `B`, et `C` sont capturées par `_backward`. Cela signifie que `_backward` a accès aux instances spécifiques de `self` (le noeud sur lequel l'opération a été appelée) et `B` (le noeud passé en argument à l'opération) qui étaient présentes au moment où l'opération a été effectuée. De plus, `C` est le nouveau noeud résultant de l'opération, et `_backward` est stocké dans cet objet `C`. C'est pourquoi vous pouvez utiliser `self`, `B`, et `C` dans votre fonction `_backward` pour calculer correctement les gradients lors de la rétropropagation.

Voici un petit exemple pour illustrer comment `self` et `B` sont capturés par la fermeture `_backward` dans le contexte de la méthode `__add__` :

```python
class Noeud:
    def __init__(self, valeur):
        self.valeur = valeur
        self.grad = 0  # Gradient initialisé à 0
        self._backward = lambda: None  # Fonction backward vide par défaut

    def __add__(self, autre):
        resultat = Noeud(self.valeur + autre.valeur)  # Crée un nouveau noeud avec la somme des valeurs

        def _backward():
            # La fermeture '_backward' capture 'self' et 'autre' ici
            self.grad += resultat.grad  # Met à jour le gradient de 'self'
            autre.grad += resultat.grad  # Met à jour le gradient de 'autre'
        
        resultat._backward = _backward  # Stocke la fonction '_backward' dans le nouveau noeud résultant

        return resultat
```

Quand on appelle `backward()` sur le noeud résultant, le processus de rétropropagation déclenche les fonctions `_backward` stockées dans chaque noeud, en commençant par le noeud de sortie et en remontant à travers le graphe de calcul. Chaque fonction `_backward` exécute le code qui a été défini lors de sa création, utilisant les instances de `self` et `B` (et éventuellement `C`) qui ont été capturées (c'est-à-dire mémorisées) par la fermeture. Cela permet de calculer et d'accumuler correctement les gradients à travers le graphe de calcul.

In [238]:
a = Noeud(3)
b = Noeud(4)

c = a * b
d = c * b

In [239]:
d.backward()
a.grad

16

## Implémentation de la Backpropagation

#### Extension à de nouvelles structures de données

Au lieu de nous limiter uniquement à des valeurs scalaires, nous généralisons notre approche aux structures de données plus complexes. En particulier, le gradient d'un noeud par rapport à ses parents, noté `dA`, est représenté par une **matrice jacobienne** dans le cas général. Cette généralisation nous permet de manipuler efficacement des gradients pour des opérations impliquant des vecteurs, des matrices, ou des tenseurs de dimensions supérieures.

#### Règles de Calcul du Gradient

Le calcul des gradients varie en fonction de l'opération effectuée sur les noeuds. Voici les formules générales pour les opérations courantes :

- **Pour une somme de deux noeuds** : La somme est une opération qui ne modifie pas le gradient. Ainsi, pour la somme de deux noeuds, nous avons :

  $
  \frac{\partial C}{\partial A} = \frac{\partial C}{\partial B} = \mathbf{dC}
  $

  où $\mathbf{dC}$ représente le gradient de la sortie par rapport à chacun des noeuds entrants.

- **Pour un produit de deux noeuds** : Le produit matriciel implique une interaction plus complexe entre les gradients. Les règles de dérivation pour un produit matriciel sont les suivantes :

  $
  \frac{\partial C}{\partial A} = \mathbf{dC} \cdot B^\top
  $
  
  $
  \frac{\partial C}{\partial B} = A^\top \cdot \mathbf{dC}
  $

  où $A^\top$ et $B^\top$ désignent les transposées des matrices $A$ et $B$, respectivement.

- **Pour une transformation unaire** : Une transformation unaire $f$ appliquée à un noeud $A$ (par exemple, une fonction d'activation dans un réseau de neurones) aura un impact sur le gradient calculé selon la règle de la chaîne. Dans ce cas, le gradient est donné par :

  $
  \frac{\partial C}{\partial A} = f'(A) \cdot \mathbf{dC}
  $

  où $f'(A)$ est la dérivée de la fonction $f$ évaluée au noeud $A$, et $\mathbf{dC}$ est le gradient de la sortie par rapport à $A$.

Ces formules constituent la base du calcul automatique des gradients dans les graphes de calcul différentiables.

#### Maintenant reprenons notre code:

In [240]:
import numpy as np

In [241]:
class Noeud:
    """
    Implémentation d'un noeud dans un graphe de calcul différentiable.
    
    Cette classe permet la création d'un graphe de calcul pour effectuer des opérations 
    élémentaires (comme l'addition, la multiplication, l'exponentiation, etc.) et calculer 
    leur gradient par rapport à leurs entrées, en utilisant le mécanisme de backpropagation.
    
    Attributs:
        valeur (np.ndarray): La valeur calculée au noeud.
        parents (list): Une liste de noeuds parents dont dépend la valeur de ce noeud.
        grad (np.ndarray): Le gradient de la fonction de perte par rapport à ce noeud, initialisé à zéro.
        _backward (function): Une fonction lambda privée qui sera définie par les opérations spécifiques pour calculer le gradient par rapport à ses parents.
    
    Méthodes:
        __init__, __add__, __mul__, somme_lignes, somme_colonnes, somme, exp, clear_grads, backward: 
        Diverses opérations et mécanismes pour construire et manipuler le graphe de calcul, ainsi que pour effectuer le backward pass.
    """

    def __init__(self, valeur, parents=[]):
        """
        Initialise un nouveau noeud avec une valeur spécifique et optionnellement des parents.
        
        Args:
            valeur (np.ndarray): La valeur du noeud.
            parents (list, optional): Les noeuds parents du noeud actuel. Defaults to [].
        """
        self.valeur = valeur.astype('float64')  # Assure que la valeur est en float64 pour la précision des calculs.
        self.parents = parents  # Liste des noeuds dont dépend ce noeud.

        # Initialisation du gradient à un tableau de zéros de même forme que la valeur.
        self.grad = np.zeros_like(self.valeur)  # dA
        self._backward = lambda: None  # Fonction lambda vide pour le backward pass. Sera définie par des opérations spécifiques.

    def __add__(self, B):
        """
        Effectue l'addition de deux noeuds en prenant en compte le broadcasting.

        Args:
            B (Noeud): Le deuxième opérande de l'addition.

        Returns:
            Noeud: Un nouveau noeud résultant de l'addition de `self` et `B`.

        Note:
            Cette méthode ajuste automatiquement les dimensions des opérandes
            pour le broadcasting conformément aux règles de numpy.
        """
        # Initialise les variables pour les opérandes, permettant le broadcasting si nécessaire.
        A_2, B_2 = self, B 

        # Gère le broadcasting sur les lignes si les dimensions ne correspondent pas.
        if self.valeur.shape[0] != B.valeur.shape[0]:
            if B.valeur.shape[0] == 1:  # Cas où B est broadcasté.
                p = self.valeur.shape[0]
                B_2 = Noeud(np.ones((p, 1))) * B
            elif self.valeur.shape[0] == 1:  # Cas où self est broadcasté.
                p = B.valeur.shape[0]
                A_2 = Noeud(np.ones((p, 1))) * self
            else:  # Erreur si le broadcasting n'est pas possible.
                print('Erreur de dimension pour le broadcasting des lignes.')

        # Gère le broadcasting sur les colonnes si nécessaire.
        elif self.valeur.shape[1] != B.valeur.shape[1]:
            if B.valeur.shape[1] == 1:  # Cas où B est broadcasté.
                p = self.valeur.shape[1]
                B_2 = B * Noeud(np.ones((1, p))) 
            elif self.valeur.shape[1] == 1:  # Cas où self est broadcasté.
                p = B.valeur.shape[1]
                A_2 = self * Noeud(np.ones((1, p)))
            else:  # Erreur si le broadcasting n'est pas possible.
                print('Erreur de dimension pour le broadcasting des colonnes.')

        # Crée un nouveau noeud résultant de l'addition.
        C = Noeud(A_2.valeur + B_2.valeur, parents=[A_2, B_2])

        # Définit la fonction de backward pour le noeud résultant.
        def _backward():
            # Ajoute le gradient du noeud résultant aux gradients des parents.
            A_2.grad += C.grad
            B_2.grad += C.grad
        C._backward = _backward

        return C

    def __mul__(self, B):
        """
        Effectue la multiplication matricielle de deux noeuds.

        Args:
            B (Noeud): Le deuxième opérande de la multiplication.

        Returns:
            Noeud: Un nouveau noeud résultant de la multiplication matricielle de `self` et `B`.
        """
        # Crée un nouveau noeud résultant de la multiplication matricielle.
        C = Noeud(np.matmul(self.valeur, B.valeur), parents=[self, B])

        # Définit la fonction de backward pour ce noeud.
        def _backward():
            # Calcule et ajoute les gradients par rapport à chaque parent.
            self.grad += np.matmul(C.grad, B.valeur.T)
            B.grad += np.matmul(self.valeur.T, C.grad)
        C._backward = _backward

        return C

    
    def somme_lignes(self):
        """
        Calcule la somme des éléments de chaque ligne du noeud.

        Returns:
            Noeud: Un nouveau noeud résultant de la somme des lignes du noeud actuel.
        """
        # Multiplie le noeud actuel par un vecteur colonne de uns pour obtenir la somme des lignes.
        C = self * Noeud(np.ones((self.valeur.shape[1], 1)))
        return C

    def somme_colonnes(self):
        """
        Calcule la somme des éléments de chaque colonne du noeud.

        Returns:
            Noeud: Un nouveau noeud résultant de la somme des colonnes du noeud actuel.
        """
        # Multiplie un vecteur ligne de uns par le noeud actuel pour obtenir la somme des colonnes.
        C = Noeud(np.ones((1, self.valeur.shape[0]))) * self
        return C

    def somme(self):
        """
        Calcule la somme de tous les éléments du noeud.

        Returns:
            Noeud: Un nouveau noeud représentant la somme totale des éléments du noeud actuel.
        """
        # Effectue la somme des lignes puis la somme des colonnes du résultat pour obtenir la somme totale.
        lignes = self.somme_lignes()
        colonnes = lignes.somme_colonnes()
        somme_finale = colonnes
        return somme_finale

    def exp(self):
        """
        Applique l'exponentielle à chaque élément du noeud.

        Returns:
            Noeud: Un nouveau noeud résultant de l'application de l'exponentielle aux éléments du noeud actuel.
        """
        # Crée un nouveau noeud avec l'exponentielle de la valeur du noeud actuel.
        C = Noeud(np.exp(self.valeur), parents=[self])

        # Définit la fonction de backward pour ce noeud.
        def _backward():
            # Calcule le gradient par rapport au noeud actuel en utilisant la dérivée de l'exponentielle.
            self.grad += np.multiply(C.valeur, C.grad)
        C._backward = _backward

        return C

    
    def clear_grads(self):
        """
        Réinitialise les gradients à 0 (en prenant en compte que nos gradients ne sont plus des floats)

        Returns: 
            Rien du tout
        """
        a_visiter = self.parents.copy()
        
        while not len(a_visiter) == 0:
            s = a_visiter.pop()
            s.grad = np.zeros_like(s.valeur) 
        
        for parent in s.parents:
            if parent not in a_visiter:
                a_visiter.append(parent)

    def backward(self):
        # Remettre à 0 les gradients
        self.clear_grads()

        dejaVu = set()
        L = []

        def parcours(s):
            dejaVu.add(s)
            
            for parent in s.parents:
                if parent not in dejaVu:
                    parcours(parent)

                # fin traitement de s
            L.append(s)
                
        parcours(self)

        self.grad = np.array([[1]]) # Il s'agit du gradient de la sortie par rapport à elle même, compatible avec les opérations numpy
        for s in reversed(L):
            s._backward()

Un exemple de graphe de calcul:

In [242]:
A = Noeud(np.array([[2,3],[1,0]]))
B = Noeud(np.array([[1,2], [3,4]]))
D = Noeud(np.array([[3,3], [2,2]])) # On implémentera ensuite le broadcasting 
d = Noeud(np.array([[3], [2]]))

C =  A * B
E = C + d
J = E.somme()

J.valeur # On a le même résultat selon que l'on utilise d ou D! Ceci grâce au broadcasting

array([[40.]])

In [243]:
J.backward()

In [244]:
E.grad

array([[1., 1.],
       [1., 1.]])

## Rapide rappel sur le parcours en profondeur d'un graphe

In [245]:
class Noeud:
    def __init__(self, nom, parents=None):
        self.nom = nom
        if parents is None:
            self.parents = []
        else:
            self.parents = parents

def parcours_profondeur(s, dejaVu=None, L=None): # dejaVu et L en argument pour éviter d'être réinitialisés à chaque appel récursif
    if dejaVu is None:
        dejaVu = set()
    if L is None:
        L = []

    dejaVu.add(s)
    for parent in s.parents:
        if parent not in dejaVu:
            parcours_profondeur(parent, dejaVu, L)

    L.append(s)
    return L

In [246]:
# Création d'un graphe similaire à celui utilisé dans notre exemple précédent 
noeud1 = Noeud("1")
noeud2 = Noeud("2")
noeud3 = Noeud("3")
noeud4 = Noeud("4", [noeud1, noeud2])
noeud5 = Noeud("5", [noeud3, noeud4])
noeud6 = Noeud("6", [noeud5])

# Parcours en profondeur du graphe à partir du noeud 4
L = parcours_profondeur(noeud6)
noms_parcours = [noeud.nom for noeud in L]

noms_parcours


['3', '1', '2', '4', '5', '6']